# Writing TTrees

As of now, uproot can write TTrees whose branches are basic types (integers and floating-point numbers).  

Low-level interface: writing individual TBaskets:

In [1]:
import uproot

branchdict = {"branch": "int32"}
tree = uproot.newtree(branchdict)
with uproot.recreate("example.root") as f:
    f["t"] = tree
    f["t"]["branch"].newbasket(numpy.array([1, 2, 3, 4, 5]))

Reading it back in uproot - 

In [2]:
f = uproot.open("example.root")
f["t"].array("branch")

array([1, 2, 3, 4, 5], dtype=int32)

Reading it back in ROOT -

In [3]:
import ROOT

f = ROOT.TFile.Open("example.root")
tree = f.Get("t")
treedata = tree.AsMatrix(["branch"])
print(treedata)

Welcome to JupyROOT 6.18/00
[[1.]
 [2.]
 [3.]
 [4.]
 [5.]]


You can look at the current state of the TTree when you are putting in data -

In [4]:
import uproot

branchdict = {"branch": "int32"}
tree = uproot.newtree(branchdict)
f = uproot.recreate("example.root")
f["t"] = tree
f["t"]["branch"].newbasket([1, 2, 3, 4, 5])

Read it - 

In [5]:
f["t"].array("branch")

array([1, 2, 3, 4, 5], dtype=int32)

Keep writing - 

In [9]:
f["t"]["branch"].newbasket([6, 7, 8, 9, 10])

Close the file!

In [10]:
f.close()

Just to check if it is a valid ROOT file - 

In [11]:
import ROOT

f = ROOT.TFile.Open("example.root")
tree = f.Get("t")
treedata = tree.AsMatrix(["branch"])
print(treedata)

[[ 1.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]]


## Creating the Branches

There are 2 ways to specify a branch -

In [12]:
b = "int32"
# b = ">i4"

or

In [13]:
b = uproot.newbranch("int32")
# b = uproot.newbranch(">i4")

The above method provides a lot more flexibility, letting you specify the flushsize, title of the branch and compression of the baskets along with the type of the branch.

To specify the title of the branch -

In [14]:
b = uproot.newbranch("int32", title="This is the title")

## Creating the Tree

Note that the name of the branches are not specified anywhere yet.

The next step is to create a dictionary of all the branches with the branch name as the key and the branch(or the type) as the value - 

In [15]:
branchdict = {"branch1": b, "branch2": ">i8"}

Then we create our tree with the dictionary we just created - 

In [16]:
t = uproot.newtree({"branch1": int, "branch2": numpy.int32, "branch3": uproot.newbranch(numpy.float64, title="This is the title")})

Similar to the branches, we can specify the title, the flushsize and the compression using the newtree method.

Again, like the branches we can specify the title of the TTree like - 

In [17]:
t = uproot.newtree(branchdict, title="TTree Title")

We can then write it to the file - 

In [18]:
f = uproot.recreate("demo.root")
f["t"] = t

### Writing baskets

We have 2 branches in our TTree -
* branch1
* branch2

In [19]:
# Write everything about append and extend

**Low level interface**  

If you want, you can write a basket to only 1 branch. But remember to add equal number of baskets to the other branches as well as ROOT assumes that all the branches have equal number of baskets and will not read the non-uniform baskets.

In [20]:
f["t"]["branch1"].newbasket([1, 2, 3])

Add 3 more baskets to branch2!

In [21]:
f["t"]["branch2"].newbasket([91, 92, 93])

## Compression

By default, the baskets of all the branches are compressed depending on the compression set for the file.

In [22]:
branchdict = {"branch": "int32"}
tree = uproot.newtree(branchdict)
with uproot.recreate("example.root", compression=uproot.LZMA(5)) as f:
    f["t"] = tree
    f["t"]["branch"].newbasket([1]*1000)

In the above example the baskets in the branch are compressed using LZMA with level equal to 5.

You can specify the compression of all the branches if you want it to be separate from the compression specified for the entire file.

In [23]:
branchdict = {"branch": "int32", "testbranch": "int64"}
tree = uproot.newtree(branchdict, compression=uproot.LZ4(4)) # <-- LOOK HERE
with uproot.recreate("example.root", compression=uproot.LZMA(5)) as f:
    f["t"] = tree
    f["t"]["branch"].newbasket([1]*1000)
    f["t"]["testbranch"].newbasket([2]*1000)

In the above example, the baskets in both branch "branch" and "testbranch" are compressed using LZ4 with level equal to 4.
If the user adds other objects(new tree, histogram, strings) to the ROOT file, they will still follow the compression specified in the file of LZMA with level equal to 5.

You can also specify the compression of each branch individually.

In the "Creating the Branches" section, we spoke about the compression option if one is using the newbranch interface to create the branch.

In [24]:
b1 = uproot.newbranch("i4", compression=uproot.ZLIB(5))
b2 = uproot.newbranch("i8", compression=uproot.LZMA(4))
b3 = uproot.newbranch("f4")

branchdict = {"branch1": b1, "branch2": b2, "branch3": b3}
tree = uproot.newtree(branchdict, compression=uproot.LZ4(4))
with uproot.recreate("example.root", compression=uproot.LZMA(5)) as f:
    f["t"] = tree
    f["t"]["branch1"].newbasket([1]*1000)
    f["t"]["branch2"].newbasket([2]*1000)
    f["t"]["branch3"].newbasket([3]*1000)

In the above example, the baskets in branch "branch1" are compressed using ZLIB with level equal to 5.  
The baskets in branch "branch2" are compressed using LZMA with level equal to 4.  
The baskets in branch "branch3" follow the compression set using the newtree interface and is using LZ4 with level equal to 4.  